Just generate 2048 features for each file with help of Inception V3 - to speed up "head" training

In [1]:
import torch
print (torch.cuda.device_count())
print (torch.__version__)

1
1.5.0+cu101


In [0]:
import requests
from torchvision.models.inception import inception_v3
from torchvision.models.resnet import resnet50
from torch import nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from skimage.transform import resize
%matplotlib inline

from tqdm import tqdm
from imageio import imread
import PIL.Image as Image
import os
import numpy as np
from sklearn.metrics import accuracy_score, log_loss

In [3]:
# MOunting google drive with images
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! cp '/content/gdrive/My Drive/Colab Notebooks/images/test.zip' ./
! cp '/content/gdrive/My Drive/Colab Notebooks/images/train.zip' ./
!unzip -n test.zip > /dev/null
!unzip -n train.zip > /dev/null
#Uploading weights from local drive - litlle bit faster
# ! cp '/content/gdrive/My Drive/Colab Notebooks/resnet50-19c8e357.pth' ./

In [5]:
!ls ./

gdrive	sample_data  test  test.zip  train  train.zip


In [0]:
# SOme usefull functions
def transform_input(img):
    return torch.as_tensor(img.reshape([1, 299, 299, 3]).transpose([0, 3, 1, 2]), dtype=torch.float32)



In [0]:
# model = resnet50(pretrained=True, progress=True)
# state_dict = torch.load('resnet50-19c8e357.pth')
# model.state_dict = state_dict

In [8]:
model = inception_v3(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth


In [9]:
model.train(mode=False)
model.cuda()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [0]:
class Identity(torch.nn.Module):

    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [0]:
model.fc = Identity()
# model

In [12]:
#freezing everything except last layers
model.train(mode=False)
model.aux_logits = False
model.cuda()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [0]:
#get file names
train_files = os.listdir('train')
# file_num = len(train_files)

test_files = os.listdir('test')

In [14]:

len(train_files), len(test_files)

(25000, 12500)

In [0]:
def first_column(filename, mode):
    if mode == 'train':
        return int(filename.startswith("dog"))

    if mode == 'test':
        return filename.split('.')[0]

In [16]:
first_column('d.178', 'train')

0

In [0]:
def predict_params(test_files, batch_size=50, tqdm_disable=False, mode='train'):
    Y = []
    Y_pred = []
    eps = 1e-15
    imgs = np.zeros([batch_size, 299, 299, 3])
    batch_index = 0

    for fname in tqdm(test_files, disable=tqdm_disable):
        y = first_column(fname, mode)
        Y.append(y)

        img = imread(os.path.join(mode, fname))
        img = np.array(Image.fromarray(img).resize((299, 299))) / 255.
        imgs[batch_index] = img

        if batch_index == batch_size-1:
            input_tensor = torch.as_tensor(imgs.transpose([0,3,1,2]), dtype=torch.float32).cuda()

            Y_p = model(input_tensor).detach().cpu()
            # print(Y_p.dim())
            # assert Y_p.dim() == 2048, "did you forget to select first column with [:, 0]"
            # Y_p = 1/(1+torch.exp(-Y_p)) # Sigmoid returns big numbers - like 22 workaround
            Y_pred.append(Y_p.data.numpy())
            batch_index=0
            continue
        
        batch_index += 1

    Y_pred = np.concatenate(Y_pred)

    # acc = accuracy_score(Y, np.round(Y_pred))
    # log_loss = -np.mean(np.array(Y)*np.log(Y_pred+eps) + (1-np.array(Y))*np.log(1-Y_pred+eps))

    return Y, Y_pred

In [18]:
Y, params = predict_params(train_files, mode='train')

100%|██████████| 25000/25000 [07:38<00:00, 54.58it/s]


In [0]:
output = np.hstack((np.array(Y).reshape(-1, 1), params))

In [0]:
output.tofile('train_params.dat')

In [21]:
!ls -al

total 1235632
drwxr-xr-x 1 root root      4096 May 11 09:15 .
drwxr-xr-x 1 root root      4096 May 11 08:59 ..
drwxr-xr-x 1 root root      4096 May  7 16:13 .config
drwx------ 4 root root      4096 May 11 09:02 gdrive
drwxr-xr-x 1 root root      4096 May  4 16:26 sample_data
drwxr-xr-x 2 root root    286720 Sep 20  2013 test
-rw------- 1 root root 284478493 May 11 09:02 test.zip
drwxr-xr-x 2 root root    770048 Sep 20  2013 train
-rw-r--r-- 1 root root 409800000 May 11 09:15 train_params.dat
-rw------- 1 root root 569918665 May 11 09:02 train.zip


In [22]:
!zip train_params.dat.zip train_params.dat
!ls -al


  adding: train_params.dat (deflated 48%)
total 1445588
drwxr-xr-x 1 root root      4096 May 11 09:16 .
drwxr-xr-x 1 root root      4096 May 11 08:59 ..
drwxr-xr-x 1 root root      4096 May  7 16:13 .config
drwx------ 4 root root      4096 May 11 09:02 gdrive
drwxr-xr-x 1 root root      4096 May  4 16:26 sample_data
drwxr-xr-x 2 root root    286720 Sep 20  2013 test
-rw------- 1 root root 284478493 May 11 09:02 test.zip
drwxr-xr-x 2 root root    770048 Sep 20  2013 train
-rw-r--r-- 1 root root 409800000 May 11 09:15 train_params.dat
-rw-r--r-- 1 root root 214992028 May 11 09:16 train_params.dat.zip
-rw------- 1 root root 569918665 May 11 09:02 train.zip


In [0]:
# !cp train_params.dat.zip '/content/gdrive/My Drive/Colab Notebooks/images/'

In [24]:
Y, params = predict_params(test_files, mode='test')
Y[:5]

100%|██████████| 12500/12500 [03:48<00:00, 54.72it/s]


['4890', '10765', '11617', '9204', '1074']

In [0]:
params.tofile()

In [25]:
# output = np.hstack((np.array(Y).reshape(-1, 1), params))
# output.tofile('test_params.dat')
params.tofile('test_params.dat')
# np.array(Y).tofile('filenames.dat', format='%s')
!zip test_params.dat.zip test_params.dat

!cp test_params.dat.zip '/content/gdrive/My Drive/Colab Notebooks/images/'
!cp filenames.dat '/content/gdrive/My Drive/Colab Notebooks/images/'
!ls -al

  adding: test_params.dat (deflated 10%)
total 1636124
drwxr-xr-x 1 root root      4096 May 11 09:24 .
drwxr-xr-x 1 root root      4096 May 11 08:59 ..
drwxr-xr-x 1 root root      4096 May  7 16:13 .config
-rw-r--r-- 1 root root    250000 May 11 09:24 filenames.dat
drwx------ 4 root root      4096 May 11 09:02 gdrive
drwxr-xr-x 1 root root      4096 May  4 16:26 sample_data
drwxr-xr-x 2 root root    286720 Sep 20  2013 test
-rw-r--r-- 1 root root 102400000 May 11 09:24 test_params.dat
-rw-r--r-- 1 root root  92449195 May 11 09:24 test_params.dat.zip
-rw------- 1 root root 284478493 May 11 09:02 test.zip
drwxr-xr-x 2 root root    770048 Sep 20  2013 train
-rw-r--r-- 1 root root 409800000 May 11 09:15 train_params.dat
-rw-r--r-- 1 root root 214992028 May 11 09:16 train_params.dat.zip
-rw------- 1 root root 569918665 May 11 09:02 train.zip


In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/images/filenames.txt', 'w') as dst:
    for fname in Y:
        dst.write('%s\n' % (fname))

In [42]:
Y[:5]

['4890', '10765', '11617', '9204', '1074']